In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers, Model
import keras_cv
from tensorflow import keras
import tensorflow.keras.applications as keras_applications
import json
import numpy as np
from PIL import Image
from io import BytesIO
import boto3
import os

In [ ]:
#make sure tf sees GPU
tf.config.list_physical_devices()

In [ ]:
# path to data directories
train_dir = 'data/split_with_back/train'
val_dir = 'data/split_with_back/val'
test_dir = 'data/split_with_back/test'

In [ ]:
# load data
train_dataset = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=None,
    shuffle=True,
    seed=42,
    )

val_dataset = image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=None,
    shuffle=True,
    seed=42,
    )

test_dataset = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=None,
    shuffle=True,
    seed=42,
    )

#get class names
class_names = train_dataset.class_names

In [ ]:
SCALED_SIZE = 224

BATCH_SIZE = 32

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(SCALED_SIZE, SCALED_SIZE),
  layers.Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomContrast(0.2),
  keras_cv.layers.RandomShear(0.2),
])

AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
  if shuffle:
    ds = ds.shuffle(1000)
      
  # batch all datasets
  ds = ds.batch(BATCH_SIZE)

  # use buffered prefetching
  return ds.prefetch(buffer_size=AUTOTUNE)

    

In [ ]:
train_dataset = prepare(train_dataset, shuffle=True, augment=True)
val_dataset = prepare(val_dataset)
test_dataset = prepare(test_dataset)

In [ ]:
# get the base model
IMG_SHAPE = (224, 224, 3)  
base_model = keras_applications.ResNet101(input_shape=IMG_SHAPE,
                                            include_top=False, 
                                            weights='imagenet') 

base_model.trainable = False

In [ ]:
# functional api model construction
model_input = (None,None,3)

inputs = tf.keras.Input(shape=model_input)
x = resize_and_rescale(inputs)
x = data_augmentation(x)
x = base_model(x, training=False) 
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x) 
outputs = layers.Dense(len(class_names), activation='softmax')(x) 

model = Model(inputs, outputs)

In [ ]:
# compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
# create callback to keep best weights
checkpoint_cb = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy')

# start fine-tuning
history = model.fit(train_dataset,
                    epochs=100, 
                    validation_data=val_dataset,
                    callbacks=[checkpoint_cb])

# load best weights
model.load_weights('best_model.keras')

In [ ]:
# unfreeze base model
base_model.trainable = True

# refreeze all layers except for the last 5
for layer in base_model.layers[:-5]:
    layer.trainable = False

for layer in base_model.layers:
    print(layer, layer.trainable)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpoint_cb2 = ModelCheckpoint('best_model2.keras', save_best_only=True, monitor='val_accuracy')

# continue training
history_fine = model.fit(train_dataset,
                         epochs=50, 
                         validation_data=val_dataset,
                         callbacks=[checkpoint_cb2])

model.load_weights('best_model2.keras')

In [ ]:
# evaluate the model on the test set
val_loss, val_accuracy = model.evaluate(test_dataset)

print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

In [ ]:
# save model after done training
model.save('trained_models/base_model.keras')

In [ ]:
#try out a prediction on certain image similar to app
model_loaded = keras.models.load_model('trained_models/base_model.keras')

def read_file_as_image(data) -> np.ndarray:
    with Image.open(image_path) as img:
        return np.array(img)

In [ ]:
image_path = 'data/split/new_data/apple_black_rot/apple_black_rot.jpg' 
image = read_file_as_image(image_path)

img_batch = np.expand_dims(image, axis=0)  #add the batch dimension
print(img_batch.shape)

In [ ]:
prediction = model_loaded.predict(img_batch)

In [ ]:
# load class names and check predicted label
predicted_class = class_names[np.argmax(predictions[0])]
confidence = float(np.max(predictions[0]))

print(predicted_class)
print(confidence)
    

In [ ]:
#put new model to s3
AWS_DEFAULT_REGION = os.environ["AWS_DEFAULT_REGION"]
AWS_ACCESS_KEY_ID = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]

s3 = boto3.client('s3')

s3 = boto3.resource(
    service_name='s3',
    region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

for bucket in s3.buckets.all():
    print(bucket.name)


In [ ]:
# upload model to S3 bucket
s3.Bucket('modelsbucket0408').upload_file(Filename='trained_models/base_model.keras', Key='base_model.keras')

In [ ]:
# upload labels to s3 bucket
fixed_class_names = [s.replace("___", "  ->  ") for s in class_names]
fixed_class_names = [s.replace(",", "") for s in fixed_class_names]
fixed_class_names = [s.replace(",", "") for s in fixed_class_names]
fixed_class_names = [s.replace("_", " ") for s in fixed_class_names]



json_file_path = 'trained_models/base_model_class_names.json'
with open(json_file_path, 'w') as json_file:
    json.dump(fixed_class_names, json_file)

print(fixed_class_names)

s3.Bucket('modelsbucket0408').upload_file(Filename='trained_models/base_model_class_names.json', Key='base_model_class_names.json')

